In [15]:
#Remove unused files
!rm -rf sample_data

# Download and read final CSV

In [16]:
!gdown --id 1mEs91KmOOpeadFD7q66pd09eSm8Z5A4C

Downloading...
From: https://drive.google.com/uc?id=1mEs91KmOOpeadFD7q66pd09eSm8Z5A4C
To: /content/segments.zip
100% 9.21M/9.21M [00:00<00:00, 72.3MB/s]


In [17]:
!unzip -q -o *.zip -d ./
#Delete zip files
!rm -rf *.zip

#Load libs

In [18]:
import numpy as np
import pandas as pd
import glob
import math
import os
import gc
import sys

In [19]:
df = pd.read_csv("segments.csv")

In [20]:
# View the data and some cleanup

In [21]:
df.head()

,segment_id,sensor_5__variance_larger_than_standard_deviation,sensor_5__sum_values,sensor_5__abs_energy,sensor_5__mean_abs_change,sensor_5__mean_change,sensor_5__mean_second_derivative_central,sensor_5__median,sensor_5__mean,sensor_5__length,sensor_5__standard_deviation,sensor_5__variation_coefficient,sensor_5__variance,sensor_5__skewness,sensor_5__kurtosis,sensor_5__root_mean_square,sensor_5__absolute_sum_of_changes,sensor_5__longest_strike_below_mean,sensor_5__longest_strike_above_mean,sensor_5__count_above_mean,sensor_5__count_below_mean,sensor_5__last_location_of_maximum,sensor_5__first_location_of_maximum,sensor_5__last_location_of_minimum,sensor_5__first_location_of_minimum,sensor_5__ratio_value_number_to_time_series_length,sensor_5__maximum,sensor_5__minimum,sensor_5__benford_correlation,sensor_5__time_reversal_asymmetry_statistic__lag_1,sensor_5__time_reversal_asymmetry_statistic__lag_2,sensor_5__time_reversal_asymmetry_statistic__lag_3,sensor_5__c3__lag_1,sensor_5__c3__lag_2,sensor_5__c3__lag_3,sensor_5__cid_ce__normalize_True,sensor_5__cid_ce__normalize_False,sensor_5__large_standard_deviation__r_0.05,sensor_5__large_standard_deviation__r_0.1,sensor_5__large_standard_deviation__r_0.2,...,"sensor_10__fft_aggregated__aggtype_""skew""","sensor_10__fft_aggregated__aggtype_""kurtosis""",sensor_10__max_langevin_fixed_point__m_3__r_30,"sensor_10__linear_trend__attr_""pvalue""","sensor_10__linear_trend__attr_""rvalue""","sensor_10__linear_trend__attr_""intercept""","sensor_10__linear_trend__attr_""slope""","sensor_10__linear_trend__attr_""stderr""",sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_0,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_1,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_2,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_3,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_4,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_5,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_6,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_7,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_8,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_9,sensor_10__ratio_beyond_r_sigma__r_0.5,sensor_10__ratio_beyond_r_sigma__r_1,sensor_10__ratio_beyond_r_sigma__r_1.5,sensor_10__ratio_beyond_r_sigma__r_2,sensor_10__ratio_beyond_r_sigma__r_2.5,sensor_10__ratio_beyond_r_sigma__r_3,sensor_10__ratio_beyond_r_sigma__r_5,sensor_10__ratio_beyond_r_sigma__r_6,sensor_10__ratio_beyond_r_sigma__r_7,sensor_10__ratio_beyond_r_sigma__r_10,sensor_10__lempel_ziv_complexity__bins_2,sensor_10__lempel_ziv_complexity__bins_3,sensor_10__lempel_ziv_complexity__bins_5,sensor_10__fourier_entropy__bins_2,sensor_10__fourier_entropy__bins_3,sensor_10__fourier_entropy__bins_5,sensor_10__permutation_entropy__dimension_3__tau_1,sensor_10__permutation_entropy__dimension_4__tau_1,sensor_10__permutation_entropy__dimension_5__tau_1,sensor_10__query_similarity_count__query_None__threshold_0.0,sensor_10__mean_n_absolute_max__number_of_maxima_7,time_to_eruption
1001461087,1001461087,1.0,14936.0,2.432267e+09,45.146917,-0.002217,-0.000542,0.0,0.248929,60001.0,201.338130,808.816894,4.053704e+04,0.086845,2.231623,201.338284,2708815.0,76.0,66.0,29864.0,30137.0,0.508558,0.508542,0.549558,0.549541,0.026966,1639.0,-1168.0,0.994912,-1713.371223,-13603.596246,-48201.489991,7.003847e+05,5.999069e+05,4.437665e+05,73.257121,14749.451685,1.0,0.0,0.0,...,3.667322,19.070576,3230.310774,3.205835e-07,0.020863,-27.976742,0.000887,0.000173,0.035090,0.041156,0.066370,0.102055,0.114242,0.176890,0.060567,0.192992,0.095022,0.115616,0.559291,0.259996,0.114081,0.049133,0.023066,0.011400,0.001017,0.000350,0.000117,0.000000,0.028083,0.015416,0.034199,0.138228,0.220352,0.314393,1.268331,1.919485,2.625701,NaN,5541.571429,10393161
1001732002,1001732002,1.0,31312.0,3.700904e+09,40.008350,-0.001183,0.000550,0.0,0.521858,60001.0,248.3

In [22]:
df.isnull().sum()

segment_id                                                         0
sensor_5__variance_larger_than_standard_deviation                  0
sensor_5__sum_values                                               0
sensor_5__abs_energy                                               0
sensor_5__mean_abs_change                                          0
                                                                ... 
sensor_10__permutation_entropy__dimension_4__tau_1                 0
sensor_10__permutation_entropy__dimension_5__tau_1                 0
sensor_10__query_similarity_count__query_None__threshold_0.0    1211
sensor_10__mean_n_absolute_max__number_of_maxima_7                 0
time_to_eruption                                                   0
Length: 1142, dtype: int64

In [23]:
df=df.dropna(axis=1, how="any")

In [24]:
df.shape

(1211, 1125)

# Prediccion


PCA CON NORMALIZACIÓN DEL DF DE 0 A 1

In [25]:
# Import required libraries
import pandas as pd
from sklearn.decomposition import PCA

import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Input
from keras.utils import np_utils

from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.utils import class_weight

from tensorflow.keras.optimizers import SGD, Adam, Adagrad, Adadelta
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras import backend as K
from keras.models import Model
 
import itertools
import numpy as np

In [26]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_df, columns=df.columns)
print(scaled_df)

      segment_id  ...  time_to_eruption
0       0.466332  ...          0.217189
1       0.466459  ...          0.429961
2       0.466968  ...          0.769462
3       0.467241  ...          0.771055
4       0.467292  ...          0.530512
...          ...  ...               ...
1206    0.463634  ...          0.999056
1207    0.463736  ...          0.010561
1208    0.463986  ...          0.940167
1209    0.464816  ...          0.637461
1210    0.465101  ...          0.517598

[1211 rows x 1125 columns]


In [37]:
scaled_df.head()

,segment_id,sensor_5__variance_larger_than_standard_deviation,sensor_5__sum_values,sensor_5__abs_energy,sensor_5__mean_abs_change,sensor_5__mean_change,sensor_5__mean_second_derivative_central,sensor_5__median,sensor_5__mean,sensor_5__length,sensor_5__standard_deviation,sensor_5__variation_coefficient,sensor_5__variance,sensor_5__skewness,sensor_5__kurtosis,sensor_5__root_mean_square,sensor_5__absolute_sum_of_changes,sensor_5__longest_strike_below_mean,sensor_5__longest_strike_above_mean,sensor_5__count_above_mean,sensor_5__count_below_mean,sensor_5__last_location_of_maximum,sensor_5__first_location_of_maximum,sensor_5__last_location_of_minimum,sensor_5__first_location_of_minimum,sensor_5__ratio_value_number_to_time_series_length,sensor_5__maximum,sensor_5__minimum,sensor_5__benford_correlation,sensor_5__time_reversal_asymmetry_statistic__lag_1,sensor_5__time_reversal_asymmetry_statistic__lag_2,sensor_5__time_reversal_asymmetry_statistic__lag_3,sensor_5__c3__lag_1,sensor_5__c3__lag_2,sensor_5__c3__lag_3,sensor_5__cid_ce__normalize_True,sensor_5__cid_ce__normalize_False,sensor_5__large_standard_deviation__r_0.05,sensor_5__large_standard_deviation__r_0.1,sensor_5__large_standard_deviation__r_0.2,...,"sensor_10__fft_aggregated__aggtype_""centroid""","sensor_10__fft_aggregated__aggtype_""variance""","sensor_10__fft_aggregated__aggtype_""skew""","sensor_10__fft_aggregated__aggtype_""kurtosis""","sensor_10__linear_trend__attr_""pvalue""","sensor_10__linear_trend__attr_""rvalue""","sensor_10__linear_trend__attr_""intercept""","sensor_10__linear_trend__attr_""slope""","sensor_10__linear_trend__attr_""stderr""",sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_0,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_1,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_2,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_3,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_4,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_5,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_6,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_7,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_8,sensor_10__energy_ratio_by_chunks__num_segments_10__segment_focus_9,sensor_10__ratio_beyond_r_sigma__r_0.5,sensor_10__ratio_beyond_r_sigma__r_1,sensor_10__ratio_beyond_r_sigma__r_1.5,sensor_10__ratio_beyond_r_sigma__r_2,sensor_10__ratio_beyond_r_sigma__r_2.5,sensor_10__ratio_beyond_r_sigma__r_3,sensor_10__ratio_beyond_r_sigma__r_5,sensor_10__ratio_beyond_r_sigma__r_6,sensor_10__ratio_beyond_r_sigma__r_7,sensor_10__ratio_beyond_r_sigma__r_10,sensor_10__lempel_ziv_complexity__bins_2,sensor_10__lempel_ziv_complexity__bins_3,sensor_10__lempel_ziv_complexity__bins_5,sensor_10__fourier_entropy__bins_2,sensor_10__fourier_entropy__bins_3,sensor_10__fourier_entropy__bins_5,sensor_10__permutation_entropy__dimension_3__tau_1,sensor_10__permutation_entropy__dimension_4__tau_1,sensor_10__permutation_entropy__dimension_5__tau_1,sensor_10__mean_n_absolute_max__number_of_maxima_7,time_to_eruption
0,0.466332,0.0,0.458304,0.000099,0.011978,0.423852,0.631833,0.0,0.458304,0.0,0.006723,0.475366,0.000099,0.450473,0.034311,0.006722,0.011978,0.019260,0.023505,0.317067,0.682933,0.508340,0.508529,0.549763,0.549816,0.027564,0.037804,0.976695,0.994355,0.650136,0.841956,0.835809,0.506892,0.491525,0.459588,0.233749,0.012386,1.0,0.0,0.0,...,0.095180,0.133506,0.524886,0.149431,3.208250e-07,0.464976,0.673722,0.425350,0.036432,0.037576,0.047453,0.071578,0.132519,0.140654,0.187171,0.073403,0.211154,0.103380,0.122783,0.762365,0.637215,0.598036,0.602370,0.442031,0.245601,0.080263,0.047836,0.019774,0.000000,0.285258,0.139216,0.317268,0.194309,0.236720,0.226220,0.405790,0.376685,0.356827,0.157942,0.217189
1,0.466459,0.0,0.459056,0.000159,0.010018,0.425034,0.639286,0.0,0.459056,0.0,0.009239,0.473042,0.000159,0.372511,0.021121,0.009239,0.010018,0.030817,0.038835,0.3285

In [38]:
# Remove prediction data and segment id
X_train=scaled_df.drop(['time_to_eruption','segment_id', 'sensor_1__variance_larger_than_standard_deviation', 'sensor_2__variance_larger_than_standard_deviation', 'sensor_3__variance_larger_than_standard_deviation', 'sensor_4__variance_larger_than_standard_deviation'],axis=1)
#X_train=scaled_df.drop(['time_to_eruption'],axis=1)
y_train=df["time_to_eruption"]


In [35]:
X_train=X_train.dropna(axis=1, how="any")

In [39]:
X_train.shape

(1211, 1119)

In [ ]:
def root_mean_squared_error(y_true, y_pred):
      return K.sqrt(K.mean(K.square(y_pred - y_true))) 


features = 950
pca = PCA(n_components=features)
X_train_PCA = pca.fit_transform(X_train)
n_inputs = features

model = Sequential()
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Dense(1,activation='swish'))

learning_rates= [1, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01, 0.0025, 0.0001]

batch_size = 1024
epochs = 300

for n in learning_rates:
  print('learing rate: ' + str(n))
  opt=Adam(learning_rate=n)
  epochs *= int(1.5)
  model.compile(loss=root_mean_squared_error, optimizer=opt, metrics=[])
  model.fit(X_train_PCA, y_train.astype(np.float32), batch_size=batch_size, epochs=epochs, verbose=1)

model.save('./Volcan.h5', save_format='h5')

learing rate: 1
Epoch 1/300
2/2 [==============================] - 4s 68ms/step - loss: 25510126.0000
Epoch 2/300
2/2 [==============================] - 0s 69ms/step - loss: 25513616.0000
Epoch 3/300
2/2 [==============================] - 0s 69ms/step - loss: 25508252.0000
Epoch 4/300
2/2 [==============================] - 0s 65ms/step - loss: 25505062.0000
Epoch 5/300
2/2 [==============================] - 0s 58ms/step - loss: 25505224.0000
Epoch 6/300
2/2 [==============================] - 0s 65ms/step - loss: 25500938.0000
Epoch 7/300
2/2 [==============================] - 0s 77ms/step - loss: 25496156.0000
Epoch 8/300
2/2 [==============================] - 0s 55ms/step - loss: 25488148.0000
Epoch 9/300
2/2 [==============================] - 0s 54ms/step - loss: 25473906.0000
Epoch 10/300
2/2 [==============================] - 0s 59ms/step - loss: 25466218.0000
Epoch 11/300
2/2 [==============================] - 0s 56ms/step - loss: 25449986.0000
Epoch 12/300
2/2 [==================